# Refinitiv Data Library for Python

## Content layer - Pricing stream - Used as a real-time data cache
This notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) either directly from the Refinitiv Data Platform or via Refinitiv Workspace or CodeBook. The example shows how to define a Pricing stream object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a Pricing stream object that way prevents your application from sending too many requests to the platform. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be very pleased to help you. 

## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session._desktop_session.Definition object at 0x7fb86115ad10 {name='default'}>

## Retrieve data

### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [3]:
stream = rd.content.pricing.Definition(
    universe = ['GBP=', 'JPY=', 'CAD='], 
    fields = ['BID', 'ASK', 'OPEN_PRC', 'HST_CLOSE']
).get_stream()

The open method tells the Pricing stream object to subscribe to the streams of the requested instruments.

In [4]:
stream.open()

<OpenState.Opened: 1>

As soon as the open method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon / Refinitiv Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [5]:
df = stream.get_snapshot()
display(df)

,Instrument,BID,ASK
0,GBP=,1.3475,1.3477
1,JPY=,113.85,113.88
2,CAD=,1.2422,1.2426


### Get a snapshot for a subset of instruments and fields

In [6]:
df = stream.get_snapshot(
    universe = ['JPY=', 'CAD='], 
    fields = ['BID', 'ASK']
)
display(df)

,Instrument,BID,ASK
0,JPY=,113.85,113.87
1,CAD=,1.2422,1.2426


### Other options to get values from the streaming cache

#### Direct access to real-time fields

In [7]:
print('CAD/BID:', stream['CAD=']['BID'])
print('JPY/BID:', stream['JPY=']['BID'])

CAD/BID: 1.2419
JPY/BID: 113.85


#### Direct acces to a streaming instrument

In [16]:
cad = stream['CAD=']
print(cad['BID'])

1.2443


#### Iteration on fields

In [17]:
print('CAD=')
for field_name, field_value in stream['CAD=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

CAD=
	BID:  1.2442
	ASK:  1.2446
JPY=
	BID:  113.88
	ASK:  113.89


#### Iteration on streaming instruments and fields

In [18]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

GBP=
	BID:  1.3476
	ASK:  1.3481
JPY=
	BID:  113.86
	ASK:  113.89
CAD=
	BID:  1.2443
	ASK:  1.2444


### Close the stream

In [19]:
stream.close()

<OpenState.Closed: 3>

Once closed is called the Pricing stream object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 

## Close the session

In [20]:
rd.close_session()